In [6]:
import random
import syft as sy
from syft.core import utils
from syft.core.frameworks.torch import utils as torch_utils
from syft.core.frameworks import encode

from syft.core.frameworks.torch.tensor import _GeneralizedPointerTensor
from syft.spdz import spdz
from syft.core.frameworks.torch.tensor import _SPDZTensor
import torch
import torch.nn.functional as F
from torch.autograd import Variable as Var
import json

hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)
james = sy.VirtualWorker(id="james", hook=hook, is_client_worker=False)

# Data Owned By Alice

In [7]:
# create our dataset
data = sy.FloatTensor([[0,0],[0,1],[1,0],[1,1]])
target = sy.FloatTensor([[0],[0],[1],[1]])

# Model Owned By Bob

In [8]:
model = sy.zeros(2,1)

# Alice <-> Bob Share

In [9]:
data = data.fix_precision().share(alice, bob)
target = target.fix_precision().share(alice, bob)
model = model.fix_precision().share(alice, bob)

In [16]:
data = torch.FloatTensor([1,2,3]).fix_precision().share(alice,bob)
assert ((data * 0.1).get().decode() == torch.FloatTensor([0.1,0.2,0.3])).all()
assert ((data * -0.1).get().decode() == torch.FloatTensor([-0.1,-0.2,-0.3])).all()
assert ((data * 1.1).get().decode() == torch.FloatTensor([1.1,2.2,3.3])).all()

# Train Logic

In [5]:
for i in range(10):
    pred = data.mm(model)
    grad = pred - target
    update = data.transpose(0,1).mm(grad)

    model = model - update * 0.1
    loss = grad.get().decode().abs().sum()
    print(loss)
    
assert loss < 0.8

2.0
1.6000000014901161
1.2999999821186066
1.0720000118017197
0.896000012755394
0.7580000013113022
0.6480000019073486
0.5599999949336052
0.4880000054836273
0.42799999564886093
